<a href="https://colab.research.google.com/github/salonigupta1/DeepLearningProjects/blob/main/OptimizeCNNModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Create CNN Model and Optimize it using Keras Tuner

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 97 kB 3.3 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.6.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [9]:
len(train_images)

60000

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [10]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [11]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [12]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [13]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 43s]
val_accuracy: 0.9121666550636292

Best val_accuracy So Far: 0.9128333330154419
Total elapsed time: 00h 05m 39s
INFO:tensorflow:Oracle triggered exit


In [14]:
model=tuner_search.get_best_models(num_models=1)[0]

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 128)       1280      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        55344     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               3096688   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 3,154,442
Trainable params: 3,154,442
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 18s 11ms/step - loss: 0.1181 - accuracy: 0.9559 - val_loss: 0.2783 - val_accuracy: 0.9115
Epoch 5/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0813 - accuracy: 0.9702 - val_loss: 0.3097 - val_accuracy: 0.9112
Epoch 6/10
1688/1688 [==============================] - 18s 10ms/step - loss: 0.0572 - accuracy: 0.9792 - val_loss: 0.4001 - val_accuracy: 0.9132
Epoch 7/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0380 - accuracy: 0.9861 - val_loss: 0.4101 - val_accuracy: 0.9088
Epoch 8/10
1688/1688 [==============================] - 18s 10ms/step - loss: 0.0302 - accuracy: 0.9894 - val_loss: 0.4577 - val_accuracy: 0.9097
Epoch 9/10
1688/1688 [==============================] - 18s 10ms/step - loss: 0.0248 - accuracy: 0.9915 - val_loss: 0.4940 - val_accuracy: 0.9118
Epoch 10/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0185 - accuracy: 0.9934 - val_loss: 0.5717 